In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.0L5R79FRJa/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.iFrRQQ6mqt/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.YVwZWSBPSf/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [ ]:
!wget https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!chmod 777 chromedriver
!mv chromedriver /usr/bin/

--2024-05-25 11:56:59--  https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 172.253.115.207, 172.253.122.207, 172.253.63.207, ...
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|172.253.115.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5795569 (5.5M) [application/zip]
Saving to: ‘chromedriver_linux64.zip.1’

chromedriver_linux6 100%[===================>]   5.53M  --.-KB/s    in 0.1s    

2024-05-25 11:56:59 (44.3 MB/s) - ‘chromedriver_linux64.zip.1’ saved [5795569/5795569]

Archive:  chromedriver_linux64.zip
  inflating: chromedriver            


In [ ]:
!apt-get update
!apt-get install chromium chromium-driver
!pip3 install selenium

import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from google.colab import files

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://deb.debian.org/debian buster-updates InRelease
Hit:4 http://deb.debian.org/debian-security buster/updates InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

In [ ]:
service = Service("/usr/bin/chromedriver")
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
# Noticia específica
info_dict = []
try:
  link = "https://elpais.com/espana/2024-03-03/el-hierro-se-resigna-a-ser-la-puerta-de-atras-de-europa.html"
  driver.get(link)
  wait = WebDriverWait(driver, 3)
  title_xpath = "//h1"
  title = wait.until(EC.visibility_of_element_located((By.XPATH, title_xpath))).text
  print(title)
  print(link)

  button_xpath = "//button[@id='comments_bar']"
  button = wait.until(EC.presence_of_element_located((By.XPATH, button_xpath)))
  driver.execute_script("arguments[0].click();", button)
  time.sleep(2)

  partial_src = "disqus.com/embed/comments/"
  iframe = wait.until(EC.presence_of_element_located((By.XPATH, f"//iframe[contains(@src, '{partial_src}')]")))
  driver.switch_to.frame(iframe)

  moreComments = True
  i = 0
  while(moreComments):
    try:
      moreCommentsDiv = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="load-more-refresh load-more-refresh--v2"]')))
      time.sleep(2)
      div_style = moreCommentsDiv.get_attribute("style")
      print(div_style)
      i = i+1
      if i < 3:
        moreCommentsButton = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a[data-action="more-posts"]')))
        driver.execute_script("arguments[0].click();", moreCommentsButton)
        time.sleep(2)
      else:
        moreComments = False
    except Exception as e:
      print(e)
      moreComments = False
      continue

  comentarios = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='post-message ']")))
  print("Total comments: ", len(comentarios))
  for comentario in comentarios:
    info_dict.append({
      'link': link,
      'title': title,
      'comment': comentario.text,
      'racism': 0 # Inicializamos a 0 por defecto
    })
except Exception as e:
  print(e)

driver.quit()

import datetime
fileName = 'comentarios_el_pais_' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + '.csv'
df = pd.DataFrame(info_dict)
df.to_csv(fileName, encoding='utf-8-sig', sep='*')

files.download(fileName)

El Hierro se resigna a ser la puerta de atrás de Europa
https://elpais.com/espana/2024-03-03/el-hierro-se-resigna-a-ser-la-puerta-de-atras-de-europa.html

display: none;
display: none;
Total comments:  18


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>